#1 Introducción

#2 Armado del ambiente

In [40]:


codigo = """
#include <string.h>
#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>

int main(int argc, char** argv)
{
  int rank;
  int cantidadProcesos;
  int tag = 1;
  char mensaje[100];

  char * parametro = argv[1];
  int orden = atoi(parametro);
  
  int matriz1[orden][orden];
  int matriz2[orden][orden];
  int matrizResultado[orden][orden];

  
  MPI_Status status;

  MPI_Init(NULL,NULL);

  MPI_Comm_rank(MPI_COMM_WORLD,&rank);
  MPI_Comm_size(MPI_COMM_WORLD,&cantidadProcesos);

  if(rank!=0)
  {
    int dest = 0;
    sprintf(mensaje,"hola mundo soy el thread %d de %d",rank,cantidadProcesos);
    MPI_Send(&mensaje,strlen(mensaje)+1,MPI_CHAR,dest,tag,MPI_COMM_WORLD);
  }
  else
  {
    for(int i=1;i<cantidadProcesos;i++)
    {
      MPI_Recv(&mensaje,100,MPI_CHAR,i,tag,MPI_COMM_WORLD,&status);
      printf("%s\\n",mensaje);
    }
  }

  MPI_Finalize();
}

"""

file = open('hello.c', 'w')
file.write(codigo)
file.close()


In [41]:
! mpicc -o hello hello.c

In [42]:
#@title Parametros: { vertical-output: true, display-mode: "both" }
ordenDeMatriz =  1#@param {type:"number"}
!mpirun --allow-run-as-root -n 3 hello ordenDeMatriz

hola mundo soy el thread 1 de 3
hola mundo soy el thread 2 de 3


#3 Desarrollo

#4 Tabla de pasos

#5 Conclusiones

#6 Bibliografia